In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import RobustScaler, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LassoCV, Lasso, LinearRegression, RidgeCV, Ridge
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.feature_selection import SelectKBest, f_regression


In [3]:
df = pd.read_csv('train_poly.csv')

# Splitting the data into features and target variable
X = df.drop(['NO2_tomorrow', 'Datum', 'Street', 'Unnamed: 0'], axis=1)  # Excluding target and categorical columns
y = df['NO2_tomorrow']

In [16]:
# Standardisierung der Features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Aufteilung in Trainings- und Testdatensätze
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Initialisierung von LassoCV
lasso_cv = LassoCV(alphas=None, cv=10, max_iter=2000000000)

# Training des LassoCV-Modells und Finden des optimalen Alpha
lasso_cv.fit(X_train, y_train)
optimal_alpha = lasso_cv.alpha_
print('Optimales Alpha:', optimal_alpha)

# Training des Lasso-Modells mit dem optimalen Alpha-Wert
lasso = Lasso(alpha=optimal_alpha)
lasso.fit(X_train, y_train)

# Identifizierung der wichtigen Merkmale
important_features = [feature for feature, coef in zip(df.drop('target', axis=1).columns, lasso.coef_) if coef != 0]
print('Wichtige Merkmale:', important_features)


Optimales Alpha: 0.014526031733300162


/Users/jonahnguyen/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.569e+03, tolerance: 2.107e+02
  model = cd_fast.enet_coordinate_descent(


KeyError: "['target'] not found in axis"

In [8]:
# Angenommen, selected_features und df sind bereits definiert
X_selected = df[selected_features]

# Standardisierung der ausgewählten Features
scaler = StandardScaler()
X_selected_scaled = scaler.fit_transform(X_selected)

# Aufteilung in Trainings- und Testdatensätze
X_train, X_test, y_train, y_test = train_test_split(X_selected_scaled, y, test_size=0.2, random_state=42)

# Initialisierung von RidgeCV
# Hier können Sie die Alphas anpassen, um verschiedene Regularisierungsstärken zu testen
ridge_cv = RidgeCV(alphas=np.logspace(-6, 6, 13), cv=10)

# Training des RidgeCV-Modells und Finden des optimalen Alpha
ridge_cv.fit(X_train, y_train)
optimal_alpha_ridge = ridge_cv.alpha_
print('Optimales Alpha für Ridge:', optimal_alpha_ridge)

# Training des Ridge-Modells mit dem optimalen Alpha-Wert
ridge = Ridge(alpha=optimal_alpha_ridge)
ridge.fit(X_train, y_train)

# Training des Ridge-Modells mit dem optimalen Alpha-Wert
ridge = Ridge(alpha=optimal_alpha_ridge)
ridge.fit(X_train, y_train)


# Da Ridge nicht direkt zur Feature-Auswahl verwendet wird, betrachten wir die Koeffizienten
ridge_coefficients = ridge.coef_
threshold_value = np.median(np.abs(ridge_coefficients))

# Sie können die Koeffizienten ausdrucken, um ihre Größe zu sehen
print('Koeffizienten des Ridge-Modells:', ridge_coefficients)

# Optional: Sie können die Merkmale mit den größten absoluten Koeffizienten auswählen
important_features_from_ridge = [feature for feature, coef in zip(selected_features, ridge_coefficients) if abs(coef) > threshold_value]
print('Wichtige Merkmale aus Ridge (basierend auf einem Schwellenwert):', important_features_from_ridge)


Optimales Alpha für Ridge: 1e-06
Koeffizienten des Ridge-Modells: [ 1.00315516e+01  2.65906653e+03  2.41397689e+02 -9.99287233e+01
  1.74421095e+02 -6.01198039e+03 -4.05304259e+01  4.05772990e+02
 -1.61469538e+01  3.04224563e+00 -2.14500310e+02 -4.16684078e+00
  9.66794472e+01  3.54613657e+00  1.17489072e+00 -1.99785330e+00
 -1.76903621e+02  3.35125840e+03  4.00836569e+01 -4.09817965e+02]
Wichtige Merkmale aus Ridge (basierend auf einem Schwellenwert): ['NO', 'TotalDailyTraffic NO', 'TotalDailyTraffic PM10', 'NO PM10', 'NO p', 'NO Hr', 'TotalDailyTraffic NO p', 'NO PM10 p', 'NO p^2', 'NO p Hr']


In [9]:
# Beschränken Sie den DataFrame auf die durch Ridge identifizierten wichtigen Features
X_important_ridge = df[important_features_from_ridge]

# Standardisierung der ausgewählten Features
scaler = StandardScaler()
X_important_ridge_scaled = scaler.fit_transform(X_important_ridge)

# Aufteilung in Trainings- und Testdatensätze
X_train_important, X_test_important, y_train, y_test = train_test_split(X_important_ridge_scaled, y, test_size=0.2, random_state=42)

# Initialisierung und Training des linearen Regressionsmodells
lin_reg = LinearRegression()
lin_reg.fit(X_train_important, y_train)

# Vorhersagen und Bewertung für das lineare Regressionsmodell
y_pred_lin_reg = lin_reg.predict(X_test_important)
mse_lin_reg = mean_squared_error(y_test, y_pred_lin_reg)
r2_lin_reg = r2_score(y_test, y_pred_lin_reg)

print("MSE (Linear Regression):", mse_lin_reg)
print("R2 Score (Linear Regression):", r2_lin_reg)


MSE (Linear Regression): 131.41594716945735
R2 Score (Linear Regression): 0.4549857484414349


In [10]:
# Beschränken Sie den DataFrame auf die durch Lasso identifizierten wichtigen Features
X_important_lasso = df[important_features_from_selected]

# Standardisierung der ausgewählten Features
scaler = StandardScaler()
X_important_lasso_scaled = scaler.fit_transform(X_important_lasso)

# Aufteilung in Trainings- und Testdatensätze
X_train_important, X_test_important, y_train, y_test = train_test_split(X_important_lasso_scaled, y, test_size=0.2, random_state=42)

# Initialisierung und Training des linearen Regressionsmodells
lin_reg = LinearRegression()
lin_reg.fit(X_train_important, y_train)

# Vorhersagen und Bewertung für das lineare Regressionsmodell
y_pred_lin_reg = lin_reg.predict(X_test_important)
mse_lin_reg = mean_squared_error(y_test, y_pred_lin_reg)
r2_lin_reg = r2_score(y_test, y_pred_lin_reg)

print("MSE (Linear Regression):", mse_lin_reg)
print("R2 Score (Linear Regression):", r2_lin_reg)


MSE (Linear Regression): 106.77552244601225
R2 Score (Linear Regression): 0.5571756494997652


In [14]:
from sklearn.linear_model import LassoCV, Lasso, RidgeCV, Ridge, LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd
import numpy as np

# Laden Sie Ihren Datensatz
df = pd.read_csv('train_poly.csv')

# Splitting the data into features and target variable
X = df.drop(['NO2_tomorrow', 'Datum', 'Street', 'Unnamed: 0'], axis=1)  # Excluding target and categorical columns
y = df['NO2_tomorrow']

# Standardisierung der Features für Lasso
scaler_lasso = StandardScaler()
X_scaled_lasso = scaler_lasso.fit_transform(X)

# Aufteilung in Trainings- und Testdatensätze für Lasso
X_train_lasso, X_test_lasso, y_train, y_test = train_test_split(X_scaled_lasso, y, test_size=0.2, random_state=42)

# Initialisierung von LassoCV
lasso_cv = LassoCV(alphas=None, cv=10, max_iter=2000000000)

# Training des LassoCV-Modells und Finden des optimalen Alpha
lasso_cv.fit(X_train_lasso, y_train)
optimal_alpha = lasso_cv.alpha_
print('Optimales Alpha (Lasso):', optimal_alpha)

# Training des Lasso-Modells mit dem optimalen Alpha-Wert
lasso = Lasso(alpha=optimal_alpha)
lasso.fit(X_train_lasso, y_train)

# Identifizierung der wichtigen Merkmale
important_features = [feature for feature, coef in zip(X.columns, lasso.coef_) if coef != 0]
print('Wichtige Merkmale (Lasso):', important_features)

# Beschränken Sie den DataFrame auf die durch Lasso identifizierten wichtigen Features
X_important = X[important_features]

# Standardisierung der ausgewählten Features für Ridge
scaler_ridge = StandardScaler()
X_important_scaled = scaler_ridge.fit_transform(X_important)

# Aufteilung in Trainings- und Testdatensätze für Ridge
X_train_ridge, X_test_ridge, y_train, y_test = train_test_split(X_important_scaled, y, test_size=0.2, random_state=42)

# Initialisierung und Training des linearen Regressionsmodells
lin_reg = LinearRegression()
lin_reg.fit(X_train_ridge, y_train)

# Vorhersagen und Bewertung für das lineare Regressionsmodell
y_pred_lin_reg = lin_reg.predict(X_test_ridge)
mse_lin_reg = mean_squared_error(y_test, y_pred_lin_reg)
r2_lin_reg = r2_score(y_test, y_pred_lin_reg)

print("MSE (Linear Regression):", mse_lin_reg)
print("R2 Score (Linear Regression):", r2_lin_reg)


Optimales Alpha (Lasso): 0.014526031733300162


/Users/jonahnguyen/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.569e+03, tolerance: 2.107e+02
  model = cd_fast.enet_coordinate_descent(


Wichtige Merkmale (Lasso): ['index', 'NO2', 'Apr', 'Aug', 'Feb', 'Jan', 'Jul', 'Jun', 'May', 'Nov', 'Oct', 'Sep', 'Weekday_0', 'Weekday_1', 'Weekday_2', 'Weekday_3', 'Weekday_4', 'Weekday_5', '1.5km_green', '1km_green', 'PM10', 'T', 'TotalDailyTraffic WVv', 'NO WVv', 'O3_max_h1 RainDur', 'PM10^2', 'PM10 T', 'PM10 WVv', 'T WD', 'p^2', 'p WVv', 'TotalDailyTraffic NO O3', 'TotalDailyTraffic NO PM10', 'TotalDailyTraffic NO WVv', 'TotalDailyTraffic O3 WVv', 'TotalDailyTraffic O3_max_h1^2', 'TotalDailyTraffic O3_max_h1 PM10', 'TotalDailyTraffic O3_nb_h1>120 PM10', 'TotalDailyTraffic O3_nb_h1>120 RainDur', 'TotalDailyTraffic O3_nb_h1>120 WVv', 'TotalDailyTraffic PM10 RainDur', 'TotalDailyTraffic PM10 T', 'TotalDailyTraffic PM10 WD', 'TotalDailyTraffic PM10 WVv', 'TotalDailyTraffic RainDur^2', 'TotalDailyTraffic T WD', 'TotalDailyTraffic T WVv', 'TotalDailyTraffic p WVv', 'TotalDailyTraffic p Hr', 'TotalDailyTraffic WD^2', 'TotalDailyTraffic WD WVv', 'TotalDailyTraffic WVv^2', 'NO^2 O3', 'NO^2